## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

В данной работе следует ответить на несколько вопросов:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Перед этим надо выполнить следующие шаги:

Шаг 1. Открыть файл и изучить информацию.

Шаг 2. Предобработка данных:
 - определить и заполнить пропущенные значения;
 - заменить вещественный тип данных на целочисленный:
 - удалить дубликаты;
 - провести лемматизацию целей кредита;
 - категоризировать данные:

Шаг 3. Ответьте на вопросы исследования.

### Описание данных
+ children — количество детей в семье
+ days_employed — общий трудовой стаж в днях
+ dob_years — возраст клиента в годах
+ education — уровень образования клиента
+ education_id — идентификатор уровня образования
+ family_status — семейное положение
+ family_status_id — идентификатор семейного положения
+ gender — пол клиента
+ income_type — тип занятости
+ debt — имел ли задолженность по возврату кредитов
+ total_income — ежемесячный доход
+ purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
# clients = pd.read_csv('/datasets/data.csv')
clients = pd.read_csv('data.csv')
clients.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [177]:

clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Таблица содержит информацию о клиентах: количество детей, семейный статус, уровень образования, трудовой стаж, уровень дохода. Таблица состоит из 21525 строк.
В столбцах с трудовым стажем 'days_employed' и ежемесячным доходом 'total_income' имеются пропуски. Эти пропуски случайные, ведь уровень дохода и трудовой стаж являются важнейшими критериями для решения о выдаче кредита, вероятно они потерялись при переносе данных в другой формат.
В столбце 'days_employed' (трудовой стаж в днях) отрицательные дробные числа.
В столбцах 'days_employed' и 'total_income' тип данных - float, их надо перевести в тип int.
В столбце 'education' одни и те же слова записаны с использованием заглавных и строчных букв, надо привести все символы к нижнему регистру.

### Шаг 2. Предобработка данных

### Обработка пропусков

 Подсчитаем количество пропусков в таблице 

In [178]:
clients.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
clients.isnull().loc[1010:1050]
# строка с пропущенными значениями
# clients.loc[1033] 

Пропуски имеются в столбцах с трудовым стажем и ежемесячным доходом. Эти пропуски случайны, т.к. эти данные очень важны при выдаче кредита. Сгруппируем таблицу по типу занятости 'income_type' и посчитаем средний доход по каждой группе занятых. Далее заполним пропуски в соответствии с типом дохода.

In [206]:
# сведем в таблицу данные о среднем доходе по каждой группе занятых 
mean_table = clients.groupby(['income_type']).agg({'total_income' : 'mean'})
mean_table

,total_income
income_type,
безработный,131339.000000
в декрете,53829.000000
госслужащий,170897.818119
компаньон,202416.971445
пенсионер,137126.973884
предприниматель,499163.000000
сотрудник,161379.779682
студент,98201.000000


In [207]:
# заполним пропуски в соответствии с типом дохода 
for inc_type in mean_table.index:
    clients.loc[clients['income_type'] == inc_type,'total_income']=clients.loc[clients['income_type'] == inc_type,'total_income'].fillna(mean_table.loc[inc_type, 'total_income'])

clients.loc[1033]
# проверим количество пропусков
clients['total_income'].isnull().sum() 
 

0

В столбце 'days_employed' имеются отрицательные значения, сначала заменим их на модуль числа, потом посчитаем средний по каждой группе занятых

In [181]:
def module(a): #Функция для вычисления модуля числа
    if a <0:
        a = a*-1
    return a
# заменим отрицательные числа их модулем
clients['days_employed'] = clients['days_employed'].apply(module)

# сведем в таблицу данные о среднем трудовом стаже по каждой группе занятых
mean_table = clients.groupby(['income_type']).agg({'days_employed' : ['mean','median']})
mean_table


days_employed               
                          mean         median
income_type                                  
безработный      366413.652744  366413.652744
в декрете          3296.759962    3296.759962
госслужащий        3399.896902    2689.368353
компаньон          2111.524398    1547.382223
пенсионер        365003.491245  365213.306266
предприниматель     520.848083     520.848083
сотрудник          2326.499216    1574.202821
студент             578.751554     578.751554

Средний трудовой стаж пенсионеров получается примерно 325909 дней = 325909/365= 892 лет, слишком много для одного человека, причем медианное значение еще больше, значит это не единичная ошибка. Возможно при переводе данных вместо трудовой стаж в днях, получили в часах и лишний раз умножили на 24 часа, попробуем разделить на 24:
325909/365/24 = 37 лет, да, пенсионеры могут иметь средний трудовой стаж  37 лет, заменим аномальные значения в столбце days_employed.
Самый большой трудовой стаж в России состовлял 80 лет.
80лет * 365дней + 20 високосных годов = 29220 дней, выше этой цифры, поделим трудовой стаж на 24.

In [210]:
def change_days(row):
    employed= row['days_employed']
    income_type = row['income_type']
    if (income_type == 'пенсионер' or income_type == 'безработный') & (employed > 29220):        
        return (employed / 24)
    else:
        return employed
    
clients['days_employed'] = clients.apply(change_days,axis=1)
mean_employed = clients.groupby(['income_type']).agg({'days_employed' : 'mean'})
mean_employed

,days_employed
income_type,
безработный,15266.500000
в декрете,3296.000000
госслужащий,3399.361016
компаньон,2111.028751
пенсионер,15207.980674
предприниматель,520.000000
сотрудник,2325.997203
студент,578.000000


In [183]:
# заполним пропуски в соответствии с типом дохода 
for inc_type in mean_employed.index:
    clients.loc[clients['income_type'] == inc_type,'days_employed']=clients.loc[clients['income_type'] == inc_type,'days_employed'].fillna(mean_employed.loc[inc_type, 'days_employed'])

clients.groupby(['income_type']).agg({'days_employed' : 'mean'})
# Проверим наличие пропусков в таблице
# clients.isnull().sum()


,days_employed
income_type,
безработный,15267.235531
в декрете,3296.759962
госслужащий,3399.896902
компаньон,2111.524398
пенсионер,15208.478802
предприниматель,520.848083
сотрудник,2326.499216
студент,578.751554


Пропуски были в столбцах с трудовым стажем 'days_employed' и ежемесячным доходом 'total_income'. Уровень дохода важный показатель для выявления влияния на выплату кредита в срок. Пропуски в данных по доходам заполнены по среднему доходу каждой группы занятых. Аномальные значения в столбце трудовой стаж заменены из предположения что при переносе данных из другого формата некоторые значения получились в часах, а не днях, после этого пропуски заменены на средние значения для каждой группы занятых. 

### Замена типа данных

Переведем 'days_employed' (трудовой стаж в днях) в класс int , т.к. это должны быть целые значения, дробные появились возможно при импорте из другого формата.

In [212]:
# Переведем 'days_employed' (трудовой стаж в днях) в класс int
clients['days_employed'] =clients['days_employed'].astype('int')
clients['days_employed'].head(10)

0     8437
1     4024
2     5623
3     4124
4    14177
5      926
6     2879
7      152
8     6929
9     2188
Name: days_employed, dtype: int64

In [213]:
# Переведем 'total_income'  в класс int
clients['total_income'] =clients['total_income'].astype('int')
clients['total_income'].head(10)

0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int64

### Вывод

Трудовой стаж в днях 'days_employed' и уровень дохода 'total_income' переведен в класс 'int' методом astype()

### Прочие ошибки в данных

Посмотрим какие значения встречаются в столбцах 'children', 'income_type', есть ли в них что-то интересное.

In [186]:
clients['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Целых 76 семей имеют по 20 детей, вряд ли все эти люди обратились в один банк за кредитом, скорее всего тут ошибка и детей все же 2, еще заменим -1 на 1.

In [187]:
# заменим количество детей с 20 на 2
clients.loc[clients['children'] == 20, 'children'] = 2 
clients['children'].value_counts()

 0    14149
 1     4818
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

In [188]:
clients['children'] = clients['children'].apply(module) #воспользуемся функцией для вычисления модуля числа
clients['children'].value_counts() # проверим значения 

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Обработка дубликатов

В столбце 'education' одни и те же слова записаны с использованием заглавных и строчных букв,
надо привести все символы к нижнему регистру.

In [189]:
clients['education'] = clients['education'].str.lower()
clients['education'].value_counts()


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [190]:
# приведем  все символы в столбце 'family_status' к нижнему регистру
clients['family_status']=clients['family_status'].str.lower()
clients['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [214]:
clients['purpose'].value_counts()

жилая_недвижимость           7622
автомобиль                   4306
образование                  4013
свадьба                      2324
строительство                1878
коммерческая_недвижимость    1311
Name: purpose, dtype: int64

#### Проверим наличие полных дубликатов

In [215]:
clients.duplicated().sum()

242

In [216]:
clients = clients.drop_duplicates().reset_index(drop=True)
clients.duplicated().sum()

0

### Вывод

В столбце 'education' строки содержали строчные и заглавные буквы, теперь все строки приведены к нижнему регистру. Строки в столбце 'family_status'также приведены к нижнему регистру, хоть там и не было дублей. В столбце 'purpose' очень много вариантов формулировок целей кредита, избавится от дублей поможет лемматизация.

### Лемматизация

Выделим основные цели получения кредита: свадьба, жилая недвижимость, оно же жилье, комерческая недвижимость, строительство, ремонт, автомобиль, образование. Сначала сведем слова в столбце 'purpose' к лемме с помощью библиотеки pymystem3 с функцией лемматизации на русском языке.


In [217]:
from pymystem3 import Mystem
m = Mystem()

lemma_data = clients['purpose'].apply(m.lemmatize)
lemma_data.value_counts()

[жилой, _, недвижимость, \n]           7488
[автомобиль, \n]                       4272
[образование, \n]                      3964
[свадьба, \n]                          2306
[строительство, \n]                    1871
[коммерческий, _, недвижимость, \n]    1311
Name: purpose, dtype: int64

Теперь напишем функцию, которая рапределит все лемматизированные строки по нескольким категориям: жилая_недвижимость, автомобиль, образование, свадьба строительство, коммерческая_недвижимость. Если встречается слово "жилье" или просто "недвижимость", отнесем это к категории жилая недвижимость, если одновременно слова "комерческий" и "недвижимость, то к категории "коммерческая_недвижимость". Заменим значения в таблице 'data'

In [218]:

def lemma_purpose(messages):
    if 'строительство' in messages:
        return 'строительство'
    if 'жилье' in messages:
        return 'жилая_недвижимость'
    
    if ('недвижимость' in messages)  and ('коммерческий' in messages):
            return 'коммерческая_недвижимость'
    if 'недвижимость' in messages:
        return 'жилая_недвижимость'
     
    if 'автомобиль' in messages:
        return 'автомобиль'
    if 'образование' in messages:
        return 'образование'
    if 'свадьба' in messages:
        return 'свадьба'
    
    
purpose_new = lemma_data.apply(lemma_purpose)
purpose_new
# print(purpose_new.value_counts())
clients['purpose'] = purpose_new
clients['purpose'].value_counts()


жилая_недвижимость           7488
автомобиль                   4272
образование                  3964
свадьба                      2306
строительство                1871
коммерческая_недвижимость    1311
Name: purpose, dtype: int64

### Вывод

В результате лемматизации цели кредита теперь распределены по 6 категориям:  жилая_недвижимость, автомобиль, образование, свадьба, строительство, коммерческая_недвижимость

### Категоризация данных

Создадим словари для столбцов 'education' и 'family_status' 

In [219]:
education_dict = clients[['education',  'education_id']] # словарь для 'education'
education_dict = education_dict.drop_duplicates().reset_index(drop=True) #уберем дубликаты 
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [220]:
family_status_dict = clients[['family_status',  'family_status_id']] # словарь для 'family_status'
family_status_dict =family_status_dict.drop_duplicates().reset_index(drop=True) #уберем дубликаты 
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


#### Разделим уровень дохода на четыре равные группы. Для этого воспользуемся коммандой quantile() для значений в  25%, 50% и 75%

In [221]:
total_income_dict = clients['total_income']

quantile25 =  clients['total_income'].quantile(.25)
quantile50 =  clients['total_income'].quantile(.5)
quantile75=  clients['total_income'].quantile(.75)
quantile_income = [clients['total_income'].min(), quantile25, quantile50, quantile75, clients['total_income'].max()]
quantile_income

[20667, 106983.75, 151425.0, 202417.0, 2265604]

In [223]:
total_income_dict = clients
print(clients['total_income'].max())
def total_income_level(money):    
    if money < quantile25:        
        return ('0%')
    if money < quantile50:
        return ('25%')
    if money < quantile75:    
        return '50%'       
    return '75%'



clients['quantile_income'] = total_income_dict['total_income'].apply(total_income_level)
total_income_dict = total_income_dict[['total_income','quantile_income', 'debt']]

# total_income_dict['quantile_income']
total_income_dict.head(10)
    



2265604


,total_income,quantile_income,debt
0,253875,75%,0
1,112080,25%,0
2,145885,25%,0
3,267628,75%,0
4,158616,50%,0
5,255763,75%,0
6,240525,75%,0
7,135823,25%,0
8,95856,0%,0
9,144425,25%,0


### Вывод

Для удобства восприятия создали словари для 'education', 'family_status',  а также разделили уровень дохода на четыре уровня с равным количеством наблюдений.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Получим сводную таблицу из таблицы 'children_data' по нужным столбцам - количество детей ('children') и наличие задолженности по возврату кредитов ('debt') и посчитаем количество случаев для (debt = 1) и (debt = 0).

Для сводной таблицы добавим для наглядности два столбца  - процент возврата (refund) и не возврата (no_refund) в срок от общего числа выданных кредита в зависимости от наличия детей.

In [200]:
#сводная таблица по наличию детей ('children_have') и наличие задолженности по возврату кредитов ('debt')
data_pivot = clients.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count')
data_pivot['no_refund'] = 100 * data_pivot[1] / (data_pivot[0] +data_pivot[1])
data_pivot['refund'] =  100 * data_pivot[0] / (data_pivot[0] +data_pivot[1])

data_pivot


debt,0,1,no_refund,refund
children,,,,
0,13028.0,1063.0,7.543822,92.456178
1,4410.0,445.0,9.165808,90.834192
2,1926.0,202.0,9.492481,90.507519
3,303.0,27.0,8.181818,91.818182
4,37.0,4.0,9.756098,90.243902
5,9.0,NaN,NaN,NaN


### Вывод

У семей с 5 детьми нет задолженности по возврату кредита в срок, но этих случаев слишком мало для 
анализа, эту строчку учитывать не будем. По остальным случаям выше всего процент возврата у людей 
без детей, 92,5% от общего числа, чуть ниже у семей с 3 детьми - 91,8%, а вот у семей 
с 1,2 и 4 детьми - 90,2 - 90,8% возвратов в срок. Итого, выше всего процент возврата у людей 
без детей, при наличии детей процент возврата кредита в срок падает. С появлением детей расходы семьи растут, а доход на каждого члена семьи уменьшается, из-за этого могут возникнуть временные финансовые трудности.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Получим сводную таблицу data_pivot_family из таблицы 'data' по нужным столбцам - 'family_status_id' и наличие задолженности по возврату кредитов ('debt') и посчитаем количество случаев для (debt = 1) и (debt = 0). Потом соединим полученную таблицу со словарем для расшифровке статуса семьи 'family_status'
В сводную таблицу добавим  два столбца  - процент возврата (refund) и не возврата (no_refund) в срок от общего числа выданных кредита.

In [224]:

data_pivot_family = clients.pivot_table(index=['family_status_id'], columns='debt', values='dob_years', aggfunc='count')
data_pivot_family['no_refund'] = 100 * data_pivot_family[1] / (data_pivot_family[0] +data_pivot_family[1])
data_pivot_family['refund'] =  100 * data_pivot_family[0] / (data_pivot_family[0] +data_pivot_family[1])

# соединим со словарем по 'family_status_id'
data_pivot_family = data_pivot_family.merge(family_status_dict, on='family_status_id', how='left')
data_pivot_family


,family_status_id,0,1,no_refund,refund,family_status
0,0,11220,930,7.654321,92.345679,женат / замужем
1,1,3738,388,9.403781,90.596219,гражданский брак
2,2,886,63,6.638567,93.361433,вдовец / вдова
3,3,1108,85,7.124895,92.875105,в разводе
4,4,2520,274,9.806729,90.193271,не женат / не замужем


### Вывод

Отсутствие задолженности по кредиту выше у вдовцов - 93.4%, у женатых и разведенных чуть ниже - 92.5% и 92,9% соответственно, самый низкий у неженатых и состоящих в гражданско м браке - 90.2% и 90.7% соответственно. 

#### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Получим сводную таблицу total_income_pivot для квантилей уровней дохода, полученных ранее при категоризации данных, из таблицы с уровнем дохода total_income_dict по нужным столбцам - уровень дохода ('level_income') и наличие задолженности по возврату кредитов ('debt') и посчитаем количество случаев для (debt = 1) и (debt = 0).
В сводную таблицу добавим  два столбца  - процент возврата (refund) и не возврата (no_refund) в срок от общего числа выданных кредита.

In [202]:
# квантили вместе с минимумом и максимом
quantile_income


[20667, 107623.0, 151887.0, 202417.0, 2265604]

Уровни дохода:
0%  -  (20.6 - 107,6)тыс.руб
25% -  (107,6 - 151,9)тыс.руб
50% -  (151,9 - 202,4)тыс.руб
75% -  (202,4 - 2 265,6)тыс.руб

In [203]:
total_income_pivot = total_income_dict.pivot_table(index=['quantile_income'], columns='debt', values= 'total_income', aggfunc='count')
total_income_pivot['refund'] = 100 * total_income_pivot[0] / (total_income_pivot[0] +total_income_pivot[1])
total_income_pivot


debt,0,1,refund
quantile_income,,,
0%,4937,427,92.039523
25%,4888,475,91.143017
50%,4853,464,91.273274
75%,5035,375,93.068392


### Вывод

Отсутствие задолженности по выплате кредита в срок выше у тех, чей  доход попал в четвертую категорию заемщики с доходом выше 202,4 тыс.руб - 92,9% возвратов кредита в срок, затем идут люди с минимальными доходами (от 20,7 до 107,6 тыс.руб) - 92,0% возвратов кредита в срок, потом заемщики из третьей категории дохода  (151,9 - 202,4 тыс.руб) - 91,2%, и чуть ниже с доходами (107,6 - 151,9 тыс.руб) - 91,1%. Таким образом, выше вероятность возврата кредита в срок у людей с максимальными и минимальными доходами. Полагаю, в случае с максимальным доходом это связано с наличием стабильного источника высокого дохода. А в случае с минимальными доходами - с лучшими проверками на платежеспособность со стороны банковских работников.

- Как разные цели кредита влияют на его возврат в срок?

Получим сводную таблицу data_pivot_purpose по нужным столбцам - цель кредита ('purpose') и наличие задолженности по возврату кредитов ('debt') и посчитаем количество случаев для (debt = 1) и (debt = 0).
В сводную таблицу добавим  два столбца  - процент возврата (refund) и не возврата (no_refund) в срок от общего числа выданных кредита.

In [204]:
data_pivot_purpose = clients.pivot_table(index=['purpose'], columns='debt', values='dob_years', aggfunc='count')
data_pivot_purpose['no_refund'] = 100 * data_pivot_purpose[1] / (data_pivot_purpose[0] +data_pivot_purpose[1])
data_pivot_purpose['refund'] =  100 * data_pivot_purpose[0] / (data_pivot_purpose[0] +data_pivot_purpose[1])

data_pivot_purpose

debt,0,1,no_refund,refund
purpose,,,,
автомобиль,3903,403,9.359034,90.640966
жилая_недвижимость,7083,539,7.071635,92.928365
коммерческая_недвижимость,1212,99,7.551487,92.448513
образование,3643,370,9.220035,90.779965
свадьба,2138,186,8.003442,91.996558
строительство,1734,144,7.667732,92.332268


### Вывод

Процент возврата кредита в срок ниже, когда цель - покупка автомобиля (90,6%) или образование (90,8%). Лучше всего с возвратом кредита в срок, когда цель - покупка жилой недвижимости (92,9%), коммерческой недвижимости (92,5%), строительство недвижимости(92,3%) возможно, люди взявшие кредит лучше оценивают свои возможности и шансы всегда вовремя платить по кредиту. Если цель кредита - свадьба, процент возврата кредита в срок также достаточно высок - 92%

### Шаг 4. Общий вывод

В результате проделанной работы, было сделано следующее:
 - Заполнены пропуски в данных по доходам заполнены по среднему доходу каждой группы занятых. Аномальные значения в столбце трудовой стаж заменены из предположения, что при переносе данных из другого формата некоторые значения получились в часах, а не днях, после этого пропуски заменены на средние значения для каждой группы занятых;
 - удалены дубликаты, исправлены ошибки в данных
 - проведена лемматизация целей кредита


#### Были установлены следующие закономерности:
 - Процент возврата кредита в срок выше у людей без детей.
 - Отсутствие задолженности по кредиту выше у вдовцов, у женатых и разведенных чуть ниже, самый низкий у неженатых и состоящих в гражданском браке.
 - Вероятность возврата кредита в срок выше у людей с самыми высоким и самыми низким уровнем дохода
 - Процент возврата кредита в срок ниже, когда цель - покупка автомобиля или образование. Лучше с возвратом кредита в срок обстоит, когда цель - покупка жилой недвижимости , коммерческой недвижимости строительство недвижимости и свадьба.